In [30]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import torch 

In [31]:
df = pd.read_csv("For DeepLearning.csv")
df.drop(columns=["Unnamed: 0"] , inplace=True)

In [32]:
# from nltk.corpus import stopwords
# stop_words = stopwords.words("english")
# # stop_words

In [33]:
from nltk.corpus import wordnet
def get_wordnet_pos(tag):
    if tag.startswith("J"):
        return wordnet.ADJ
    if tag.startswith("V"):
        return wordnet.VERB
    if tag.startswith("N"):
        return wordnet.NOUN
    if tag.startswith("R"):
        return wordnet.ADV
    return wordnet.NOUN

In [34]:
# # This is for the classical nlp approach 
# import re
# from nltk.stem import SnowballStemmer, WordNetLemmatizer
# from nltk import pos_tag 
# lematizer = WordNetLemmatizer()
# stemmer = SnowballStemmer("english")
# end = df.index.stop
# for i in range(end) :
#     sentence = re.sub(r"\s+", " ", re.sub(r"[^a-zA-Z0-9\s]", " ", re.sub(r"<[^>]+>"," ", df.iloc[i,0]))).strip().lower()
#     sentence = sentence.split()
#     cleaned_sentence = [word for word in sentence if word.lower() not in stop_words and word!="" ]
#     pos_tags = pos_tag(cleaned_sentence)
#     cleaned_sentence = [lematizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]
#     df.iloc[i,0] = cleaned_sentence
    
    

In [35]:
df =  pd.read_csv("IMDB Dataset.csv")

In [36]:
df_2 =  pd.read_csv("IMDB Dataset.csv")


In [37]:
df.shape

(50000, 2)

In [38]:
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
for i in tqdm(range(df.index.stop)) : 
    embeddings = model.encode(df.iloc[i,0], convert_to_tensor=True)
    df.iloc[i,0] = embeddings



c:\Users\Vedant\anaconda3\envs\Pytorch_cuda_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [ ]:
df.head()

,review,sentiment
0,"tensor([ 3.0099e-02, 5.0418e-02, -7.6095e-02,...",NaN
1,"tensor([-1.2202e-02, 5.1961e-02, -1.8610e-02,...",NaN
2,"tensor([ 1.4258e-02, -7.9138e-02, 3.3383e-04,...",NaN
3,"tensor([-4.1720e-02, 1.0464e-02, -4.8342e-02,...",NaN
4,"tensor([-3.1675e-02, 6.4242e-03, -3.4045e-02,...",NaN


In [ ]:
df_2["sentiment"] = df_2["sentiment"].map({"positive" :1 , "negative" :0})

In [ ]:
X = np.vstack(df["review"].apply(lambda x: x.detach().cpu().numpy()))
y = df_2["sentiment"].values

AttributeError: 'str' object has no attribute 'detach'

In [ ]:
y.shape, X.shape

((50000,), (50000, 384))

In [ ]:
# df.to_csv("For DeepLearning.csv")

In [ ]:
import torch.nn as nn 

class Classifier_modelV0(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.layer1 = nn.Linear(in_features=384 , out_features=1)
        

    def forward(self , x) :
        return self.layer1(x)
    

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
from sklearn.model_selection import train_test_split
X = torch.from_numpy(X).type(torch.float32)
X = torch.nn.functional.normalize(X, p=2, dim=1)
y = torch.from_numpy(y).type(torch.float32)
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size=0.25 )

In [ ]:
from torch.optim import Adam
model_0 = Classifier_modelV0().to(device)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = Adam(params=model_0.parameters() , lr=0.001)

In [ ]:
# Calculate accuracy (a classification metric)
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc

In [ ]:
# torch.manual_seed(42)

# # Set the number of epochs
# epochs = 10000

# # Put data to target device
# X_train, y_train = X_train.to(device), y_train.to(device)
# X_test, y_test = X_test.to(device), y_test.to(device)

# # Build training and evaluation loop
# for epoch in range(epochs):
#     model_0.train()

#     # 1. Forward pass (model outputs raw logits)
#     y_logits = model_0(X_train).squeeze() # squeeze to remove extra `1` dimensions, this won't work unless model and data are on same device 
#     y_pred = torch.round(torch.sigmoid(y_logits)) # turn logits -> pred probs -> pred labls
  
#     # 2. Calculate loss/accuracy
#     # loss = loss_fn(torch.sigmoid(y_logits), # Using nn.BCELoss you need torch.sigmoid()
#     #                y_train) 
#     loss = loss_fn(y_logits, # Using nn.BCEWithLogitsLoss works with raw logits
#                    y_train) 
#     acc = accuracy_fn(y_true=y_train, 
#                       y_pred=y_pred) 

#     # 3. Optimizer zero grad
#     optimizer.zero_grad()

#     # 4. Loss backwards
#     loss.backward()

#     # 5. Optimizer step
#     optimizer.step()

#     ### Testing
#     model_0.eval()
#     with torch.inference_mode():
#         # 1. Forward pass
#         test_logits = model_0(X_test).squeeze() 
#         test_pred = torch.round(torch.sigmoid(test_logits))
#         # 2. Caculate loss/accuracy
#         test_loss = loss_fn(test_logits,
#                             y_test)
#         test_acc = accuracy_fn(y_true=y_test,
#                                y_pred=test_pred)

#     # Print out what's happening every 10 epochs
#     if epoch % 10 == 0:
#         print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%")

Epoch: 0 | Loss: 0.69424, Accuracy: 47.09% | Test loss: 0.69376, Test acc: 48.49%
Epoch: 10 | Loss: 0.68870, Accuracy: 62.19% | Test loss: 0.68823, Test acc: 62.54%
Epoch: 20 | Loss: 0.68330, Accuracy: 68.01% | Test loss: 0.68284, Test acc: 67.97%
Epoch: 30 | Loss: 0.67803, Accuracy: 71.22% | Test loss: 0.67760, Test acc: 71.10%
Epoch: 40 | Loss: 0.67290, Accuracy: 72.92% | Test loss: 0.67248, Test acc: 73.00%
Epoch: 50 | Loss: 0.66789, Accuracy: 73.92% | Test loss: 0.66750, Test acc: 73.98%
Epoch: 60 | Loss: 0.66301, Accuracy: 74.46% | Test loss: 0.66265, Test acc: 74.52%
Epoch: 70 | Loss: 0.65825, Accuracy: 74.79% | Test loss: 0.65792, Test acc: 74.74%
Epoch: 80 | Loss: 0.65362, Accuracy: 75.14% | Test loss: 0.65331, Test acc: 75.14%
Epoch: 90 | Loss: 0.64911, Accuracy: 75.34% | Test loss: 0.64882, Test acc: 75.53%
Epoch: 100 | Loss: 0.64472, Accuracy: 75.54% | Test loss: 0.64445, Test acc: 75.71%
Epoch: 110 | Loss: 0.64044, Accuracy: 75.76% | Test loss: 0.64019, Test acc: 75.92%
Epo

In [ ]:

model_1 = nn.Sequential(
    nn.Linear(in_features=384, out_features=1),
    ).to(device)



In [ ]:
from torch.optim import Adam
model_0 = Classifier_modelV0().to(device)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = Adam(params=model_1.parameters() , lr=1e-3)

In [ ]:
!pip install mlflow

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.7 MB 6.7 MB/s eta 0:00:02
   ------- -------------------------------- 1.8/9.7 MB 4.8 MB/s eta 0:00:02
   ---------- ----------------------------- 2.6/9.7 MB 4.3 MB/s eta 0:00:02
   ------------ --------------------------- 3.1/9.7 MB 4.1 MB/s eta 0:00:02
   ------------------ --------------------- 4.5/9.7 MB 4.1 MB/s eta 0:00:02
   --------------------- ------------------ 5.2/9.7 MB 4.0 MB/s eta 0:00:02
   ------------------------- -------------- 6.3/9.7 MB 4.1 MB/s eta 0:00:01
   ----------------------------- ---------- 7.1/9.7 MB 4.0 MB/s eta 0:00:01
   -------------------------------- ------- 7.9/9.7 MB 4.1 MB/s eta 0:00:01
   ----------------------------------- ---- 8.7/9.7 MB 4.0 MB/s eta 0:00:01
   ---------------------------------------- 9.7/9.7 MB 4.0 MB/s  0:00:02
   ----------------------

In [ ]:
torch.manual_seed(42)

# Set the number of epochs
epochs = 1000

# Put data to target device
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

# Build training and evaluation loop
for epoch in tqdm(range(epochs)):
    model_1.train()

    # 1. Forward pass (model outputs raw logits)
    y_logits = model_1(X_train).squeeze() # squeeze to remove extra `1` dimensions, this won't work unless model and data are on same device 
    y_pred = torch.round(torch.sigmoid(y_logits)) # turn logits -> pred probs -> pred labls
  
    # 2. Calculate loss/accuracy
    # loss = loss_fn(torch.sigmoid(y_logits), # Using nn.BCELoss you need torch.sigmoid()
    #                y_train) 
    loss = loss_fn(y_logits, # Using nn.BCEWithLogitsLoss works with raw logits
                   y_train) 
    acc = accuracy_fn(y_true=y_train, 
                      y_pred=y_pred) 

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model_1.eval()
    with torch.inference_mode():
        # 1. Forward pass
        test_logits = model_1(X_test).squeeze() 
        test_pred = torch.round(torch.sigmoid(test_logits))
        # 2. Caculate loss/accuracy
        test_loss = loss_fn(test_logits,
                            y_test)
        test_acc = accuracy_fn(y_true=y_test,
                               y_pred=test_pred)

    # Print out what's happening every 10 epochs
    if epoch % 1000 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%")
print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%")

  0%|          | 8/5000 [00:00<01:10, 70.49it/s]

Epoch: 0 | Loss: 0.40033, Accuracy: 81.92% | Test loss: 0.40189, Test acc: 81.51%


 21%|██        | 1056/5000 [00:02<00:07, 502.71it/s]

Epoch: 1000 | Loss: 0.39386, Accuracy: 82.20% | Test loss: 0.39689, Test acc: 81.83%


 41%|████▏     | 2073/5000 [00:04<00:04, 597.95it/s]

...

Epoch: 3000 | Loss: 0.38954, Accuracy: 82.43% | Test loss: 0.39366, Test acc: 81.85%


 82%|████████▏ | 4086/5000 [00:07<00:01, 602.27it/s]

Epoch: 4000 | Loss: 0.38866, Accuracy: 82.38% | Test loss: 0.39309, Test acc: 81.96%


100%|██████████| 5000/5000 [00:09<00:00, 547.82it/s]

Epoch: 4999 | Loss: 0.38818, Accuracy: 82.40% | Test loss: 0.39285, Test acc: 81.90%
